# Prepare Reference Files

As an example, we use the mouse. We need to prepare its transcriptome reference sequences, including gene names, chromosome names, strands, transcription start and end positions.


For example, we can download the fasta file using martview provided by Emsembl(https://www.ensembl.org/biomart/martview/), and we choose Dataset - Mouse genes (GRCm39), and attributes including "Transcript stable ID", "cDNA sequences", "Gene name", "Transcript start (bp)", "Transcript end (bp)", "Chromosome/scaffold name", "Strand".

Then, these sequences are grouped by chromosome and strand, and reordered to generate new files according to the order of each gene on the chromosome. Meanwhile, a geneName file is generated to record each gene and its corresponding multiple transcripts.


In [36]:
! gunzip ../martquery_0328111729_201.txt.gz && mv ../martquery_0328111729_201.txt ../GRCm39.cdna.fa

In [37]:
! mkdir ../m39REF

In [38]:
from Bio import SeqIO


records = SeqIO.parse("../GRCm39.cdna.fa", "fasta")
byChr = {}
for i in records:
    chr_strand = ('.').join(i.id.split('|')[-2:])
    if chr_strand not in byChr:
        byChr[chr_strand] = []
    byChr[chr_strand].append(i)

In [ ]:
chr_strand = '16.-1'
records_list = byChr[chr_strand]


In [39]:
filenames = [i for i in set(byChr.keys()) if i.split('.')[0] 
             in ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 
                 '11', '12', '13', '14', '15', '16', '17', '18', '19', 'X', 'Y']]

with open('../m39REF/geneNames.txt', 'a') as gw:
    reFnames = []
    recSizes = []
    clstrSizes = []
    for chr_strand, records in byChr.items():
        if chr_strand not in filenames:
            continue
        chr = chr_strand.split('.')[0]
        strand = 'forward' if chr_strand.split('.')[1] == '1' else 'reverse'
        with open(f"../m39REF/chr{chr}.{strand}.cDNA.fa", "w") as w:
            all_start = []
            for i in records:
                start = int(i.id.split('|')[2])
                all_start.append(start)
            sorted_records = sorted(zip(records, all_start), key=lambda x: x[1])
            sorted_records = [record for record, _ in sorted_records]

            name2clstr = {}
            idx2clstr = {}
            clstr_cnt = 0
            for idx, i in enumerate(sorted_records):
                name = i.id.split('|')[1]
                if name not in name2clstr:
                    name2clstr[name] = clstr_cnt
                    clstr_cnt += 1
                idx2clstr[idx] = name2clstr[name]
            for idx, i in enumerate(sorted_records):
                info = i.id.split('|')
                new_info = info[0] + '|' + info[2] + '|' + info[3] + '|' + str(idx2clstr[idx])
                w.write(f'>{new_info}\n{i.seq}\n')

            for name in name2clstr:
                gw.write(f'{name}\n')
            
            clstrSizes.append(clstr_cnt)
            recSizes.append(len(sorted_records))
            reFnames.append(f"./m39REF/chr{chr}.{strand}.cDNA.fa")
            

In [41]:
print('{"' + '"\n"'.join(reFnames) + '"}')

{"./m39REF/chr16.reverse.cDNA.fa"
"./m39REF/chrX.reverse.cDNA.fa"
"./m39REF/chr7.reverse.cDNA.fa"
"./m39REF/chr2.forward.cDNA.fa"
"./m39REF/chr11.forward.cDNA.fa"
"./m39REF/chr3.reverse.cDNA.fa"
"./m39REF/chr11.reverse.cDNA.fa"
"./m39REF/chr4.forward.cDNA.fa"
"./m39REF/chr13.forward.cDNA.fa"
"./m39REF/chr6.forward.cDNA.fa"
"./m39REF/chr17.forward.cDNA.fa"
"./m39REF/chr15.forward.cDNA.fa"
"./m39REF/chr5.reverse.cDNA.fa"
"./m39REF/chr9.forward.cDNA.fa"
"./m39REF/chr7.forward.cDNA.fa"
"./m39REF/chr14.reverse.cDNA.fa"
"./m39REF/chr6.reverse.cDNA.fa"
"./m39REF/chr8.forward.cDNA.fa"
"./m39REF/chr9.reverse.cDNA.fa"
"./m39REF/chr1.reverse.cDNA.fa"
"./m39REF/chr16.forward.cDNA.fa"
"./m39REF/chrX.forward.cDNA.fa"
"./m39REF/chr4.reverse.cDNA.fa"
"./m39REF/chr3.forward.cDNA.fa"
"./m39REF/chr2.reverse.cDNA.fa"
"./m39REF/chr10.reverse.cDNA.fa"
"./m39REF/chr5.forward.cDNA.fa"
"./m39REF/chr15.reverse.cDNA.fa"
"./m39REF/chr10.forward.cDNA.fa"
"./m39REF/chr18.forward.cDNA.fa"
"./m39REF/chr1.forward.cDNA

In [42]:
print(recSizes)

[2365, 2577, 6619, 5828, 4531, 3562, 4629, 3989, 2845, 4341, 3799, 2741, 4603, 4324, 6157, 2979, 3902, 3695, 3978, 4600, 2425, 2728, 3934, 3652, 5716, 3261, 4705, 2798, 3612, 2084, 4536, 3177, 2962, 3028, 3772, 2765, 2308, 1763, 3174, 2274, 1008, 1293]


In [43]:
print(clstrSizes)

[812, 1273, 2558, 2006, 1505, 1484, 1596, 1502, 1259, 1625, 1168, 948, 1623, 1500, 2217, 1207, 1557, 1345, 1396, 1735, 826, 1365, 1424, 1450, 1849, 1289, 1706, 968, 1383, 771, 1790, 1262, 1193, 1337, 1209, 1337, 711, 651, 1283, 736, 688, 882]


# Recompile  

Finally, modify the constants in ../src/constants.cpp, including file_end=42;reFnames={...}, clstrSizes={...} and clstrSizes={...}. Then, navigate to the parent directory and recompile.

In [45]:
! mv m39REF ../ && cd .. && make

make: 'readon' is up to date.
